In [ ]:
# -
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

# -
import os
import h5py
import numpy as np

from tqdm import tqdm
from glob import glob
from pprint import pprint

from scipy.io import loadmat

from fakespikes.util import create_times
from fakespikes.util import create_psd
from alphalogical.util import find_bursts
from alphalogical.util import find_bursts2
from alphalogical.util import find_notbursts
from alphalogical.util import butter_bandpass_filter

# ----------------------------------------
def plot_grid_psd(x, m, fs, show_plot=False):
    freqs, psd = create_psd(x[m], fs)
    p = figure(plot_width=150, plot_height=150)
    p.line(freqs, np.log10(psd), color="black", alpha=1)
    p.xaxis.axis_label = 'Freq (Hz)'
    p.yaxis.axis_label = 'Log power (AU)'
    p.x_range = Range1d(1, 50)
    p.y_range = Range1d(-3, 5)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.toolbar.logo = None
    p.toolbar_location = None
    
    if show_plot:
        show(p)
    else:
        return p

# Pathing

In [ ]:
DATA_PATH = "/Users/type/Data/Smith/data"

# Load example data

In [ ]:
pattern = "*converted_ns2*"
files = glob(os.path.join(DATA_PATH, pattern))
pprint([(i, fi) for i, fi in enumerate(files)])

# Select file by index

In [ ]:
i = 42
print("Running {}".format(files[i]))

# Basic exploration of file `i`.

In [ ]:
fi = h5py.File(files[i])

In [ ]:
pprint(fi.keys())

# Create time

In [ ]:
fs = float(fi['Fs'].value)
n_samples = float(fi['nSamples'].value) 
T = n_samples / fs
print("Experiment time T {}".format(T))

times = create_times(T, 1/fs)
print("Times: {}".format(times[:20]))

# Data stats

In [ ]:
print("Data shape: {}".format(fi['data'].shape))
X = fi['data'].value

# Choose an electrode

In [ ]:
n = 20
print("Electrode, shape:".format(n))

x = X[:, n]
pprint(x.shape)

In [ ]:
tspan = (600, 800)
m = np.logical_and(times > tspan[0], times <= tspan[1]) 

p = figure(plot_width=800, plot_height=300)
p.line(times[m], x[m], color="black", alpha=0.6)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'LFP (uV)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

freqs, psd = create_psd(x[m], fs)
p = figure(plot_width=300, plot_height=300)
p.line(freqs, np.log10(psd), color="black", alpha=1)
p.xaxis.axis_label = 'Freq (Hz)'
p.yaxis.axis_label = 'Log power (AU)'
p.x_range = Range1d(1, 50)
p.y_range = Range1d(-3, 5)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Plot LFP PSDs

If n_plot is less than 95, a random subset of electrodes is displayed

In [ ]:
tspan = (200, 800)
m = np.logical_and(times > tspan[0], times <= tspan[1]) 

# -
n_plot = 20
row_length = 5
electrodes = np.arange(0, 95)
if n_plot < 95:
    np.random.shuffle(electrodes)
    electrodes = electrodes[:n_plot]

grid = []
row = []
row_count = 0
for n_e in tqdm(electrodes):
    x = X[:, n_e]
    p = plot_grid_psd(x, m, fs)
    row.append(p)
    
    row_count += 1
    if row_count > (row_length - 1):
        grid.append(row)
        
        row = []
        row_count = 0
        
show(gridplot(grid))

# Plot EEG channels

In [ ]:
tspan = (200, 800)
m = np.logical_and(times > tspan[0], times <= tspan[1]) 

# -
electrodes = np.arange(96, 96 + 8)
row_length = 4

grid = []
row = []
row_count = 0
for n_e in tqdm(electrodes):
    x = X[:, n_e]
    p = plot_grid_psd(x, m, fs)
    row.append(p)
    
    row_count += 1
    if row_count > (row_length - 1):
        grid.append(row)
        
        row = []
        row_count = 0

show(gridplot(grid))